# Cargamos la Informacion

### Carga de .txt

In [1]:
import os

# Cargamos todos los .txt
ruta = 'sagrada-main/datos/informacion'  
textos = []

for archivo in os.listdir(ruta):
    if archivo.endswith('.txt'):
        with open(os.path.join(ruta, archivo), 'r', encoding='utf-8') as f:
            textos.append(f.read())

# Tenemos 33 textos en total
len(textos)

33

### Carga de Relaciones

In [2]:
import pandas as pd

# Cargamos los archivos CSV de relaciones
ruta_relaciones = 'sagrada-main/datos/relaciones'
archivos_deseados = ['relaciones_mediante_links.csv', 'relaciones_sagrada_generadas.csv']
dfs_relaciones = []

for archivo in os.listdir(ruta_relaciones):
    if archivo in archivos_deseados:
        ruta_completa = os.path.join(ruta_relaciones, archivo)
        df = pd.read_csv(ruta_completa)
        dfs_relaciones.append(df)

# Concatenamos los 2 DataFrames de relaciones
df_relaciones = pd.concat(dfs_relaciones, ignore_index=True)

df_relaciones.head(5)

,SUJETO1,RELACION,SUJETO2
0,Daryl Andrews,Sagrada - Diseñador,Daryl is a game designer and a member of the G...
1,Adrian Adamescu,Sagrada - Diseñador,Adrian is a Game Designer and a member of theG...
2,Peter Wocken,Sagrada - Diseñador,I am the Head of Graphic Design at Pandasaurus...
3,Floodgate Games,Sagrada - Editorial,Microbadge:Floodgate Games
4,Cranio Creations,Sagrada - Editorial,Cranio Creationsis a new and creative Italian ...


### Carga de Estadísticas

In [3]:
# Cargamos los archivos CSV de estadísticas
ruta_estadisticas = 'sagrada-main/datos/estadisticas'
dfs_estadisticas = []

for archivo in os.listdir(ruta_estadisticas):
    if archivo.endswith('.csv'):
        ruta_completa = os.path.join(ruta_estadisticas, archivo)
        df = pd.read_csv(ruta_completa, sep=';')
        dfs_estadisticas.append(df)

df_estadisticas = pd.concat(dfs_estadisticas, ignore_index=True)

df_estadisticas.head(5)


,GAME STATS,GAME RANKS,PLAY STATS,COLLECTION STATS,PARTS EXCHANGE
0,Avg. Rating 7.472,Overall Rank 213 Historical Rank,"All Time Plays 287,803","Own 73,849",Has Parts 19
1,"No. of Ratings 44,408",Abstract Rank 10 Historical Rank,This Month 512,"Prev. Owned 4,914",Want Parts 19
2,Std. Deviation 1.16,Family Rank 45 Historical Rank,NaN,For Trade 606 Find For-Trade Matches,NaN
3,Weight 1.92 / 5,NaN,NaN,Want In Trade 832 Find Want-in-Trade Matches,NaN
4,"Comments 6,141",NaN,NaN,"Wishlist 9,322",NaN


### Union de todo

In [4]:
# Convertimos estadísticas a texto
textos_estadisticas = []
for _, fila in df_estadisticas.iterrows():
    texto = ' | '.join([f"{col}: {fila[col]}" for col in df_estadisticas.columns])
    textos_estadisticas.append(texto)

# Convertimos relaciones a texto
textos_relaciones = []
for _, fila in df_relaciones.iterrows():
    texto = ' | '.join([f"{col}: {fila[col]}" for col in df_relaciones.columns])
    textos_relaciones.append(texto)

documentos = textos + textos_estadisticas + textos_relaciones
print(f"Total de documentos: {len(documentos)}")


Total de documentos: 150


## Base de datos vectorial

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30
)

fragmentos = []

for doc in documentos:
    fragmentos.extend(splitter.split_text(doc))

print(f"Total de fragmentos generados: {len(fragmentos)}")


Total de fragmentos generados: 1662


In [6]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

# Inicializamos cliente local
client = chromadb.Client()

# Definimos función de embeddings
embedding_fn = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

# Creamos colección
coleccion = client.create_collection(name="base_sagrada", embedding_function=embedding_fn)

# Insertamos fragmentos
coleccion.add(
    documents=fragmentos,
    ids=[f"id_{i}" for i in range(len(fragmentos))]
)


d:\Documentos del Administrador\Desktop\Facultad\Cuartas Materias\NLP\TP3\tp3_entorno_nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def buscar_fragmentos(consulta, k=5):
    resultado = coleccion.query(
        query_texts=[consulta],
        n_results=k
    )
    return resultado['documents'][0]

# Ejemplo
buscar_fragmentos("¿Dónde está publicado el juego?")


['3. El juego se encuentra publicado en nuestro país por Devir en una edición en español, ya que el juego muestra cierta dependencia del idioma en algunas cartas. Permite partidas de 1 a 4 jugadores, con una edad mínima sugerida de 14 años y una duración aproximada de entre 30 y 45 minutos. El precio',
 '¿Son los mismos juegos? Obviamente no, pero, salvo que seáis amantes de estas mecánicas u os encante estar constantemente probando juegos distintos, lo normal será que solo os hagáis con uno de los tres (si es que pertenecéis al grupo objetivo al que están destinados). Como curiosidad, la',
 '5. Importante: si ya conoces el juego y/o sólo te interesa mi opinión sobre el mismo, puedes pasar directamente al apartado de Opinión. Los apartados Contenido y Mecánica están destinados especialmente a aquellos que no conocen el juego y prefieren hacerse una idea general de cómo funciona.',
 'la otra cara, servirá para llevar la cuenta de los puntos acumulados por los jugadores mediante las cart

## Acceso a los Datos Estadísticos

In [14]:
df_estadisticas

,GAME STATS,GAME RANKS,PLAY STATS,COLLECTION STATS,PARTS EXCHANGE
0,Avg. Rating 7.472,Overall Rank 213 Historical Rank,"All Time Plays 287,803","Own 73,849",Has Parts 19
1,"No. of Ratings 44,408",Abstract Rank 10 Historical Rank,This Month 512,"Prev. Owned 4,914",Want Parts 19
2,Std. Deviation 1.16,Family Rank 45 Historical Rank,NaN,For Trade 606 Find For-Trade Matches,NaN
3,Weight 1.92 / 5,NaN,NaN,Want In Trade 832 Find Want-in-Trade Matches,NaN
4,"Comments 6,141",NaN,NaN,"Wishlist 9,322",NaN
5,"Fans 2,234",NaN,NaN,NaN,NaN
6,"Page Views 1,852,791",NaN,NaN,NaN,NaN
7,Avg. Rating 0.000,Overall Rank ‐‐,All Time Plays 0,Own 7,Has Parts 0
8,No. of Ratings 0,NaN,This Month 0,Prev. Owned 0,Want Parts 0
9,Std. Deviation 0.00,NaN,NaN,For Trade 1 Find For-Trade Matches,NaN


In [19]:
# Para poder extraer correctamente la informacion de importancia del df de estadística, debemos corregir las columnas
# para tener tanto categoricas como numéricas

import re
import numpy as np
df_estadisticas_procesado = pd.DataFrame()

# Función para extraer nombre y valor numérico de un string
def split_stat(valor):
    if pd.isna(valor):
        return (None, None)
    # Captura nombre y valor numérico (ignora texto adicional)
    # Ej: "Avg. Rating 7.472" → nombre=Avg. Rating, valor=7.472
    # Usamos regex para buscar el primer número decimal o entero
    nombre_match = re.match(r"^[^\d]+", valor)
    nombre = nombre_match.group(0).strip() if nombre_match else valor.strip()
    
    # Extraemos primer número válido con o sin coma como separador de miles
    valor_match = re.search(r"(\d[\d,\.]*)", valor)
    if valor_match:
        valor_str = valor_match.group(1).replace(",", "")
        try:
            valor_num = float(valor_str)
        except:
            valor_num = np.nan
    else:
        valor_num = np.nan
    
    return nombre, valor_num

filas = []
for col in df_estadisticas.columns:
    for v in df_estadisticas[col]:
        nombre, valor = split_stat(v)
        if nombre is not None:
            filas.append({
                "grupo": col,
                "nombre_metric": nombre,
                "valor_num": valor
            })

df_estadisticas_procesado = pd.DataFrame(filas)

df_estadisticas_procesado["tipo"] = np.where(df_estadisticas_procesado["valor_num"].notna(), "numérica", "categórica")

df_estadisticas_procesado.head(5)

,grupo,nombre_metric,valor_num,tipo
0,GAME STATS,Avg. Rating,7.472,numérica
1,GAME STATS,No. of Ratings,44408.000,numérica
2,GAME STATS,Std. Deviation,1.160,numérica
3,GAME STATS,Weight,1.920,numérica
4,GAME STATS,Comments,6141.000,numérica


In [21]:
def extraer_info_importante(df):
    info = {}

    grupos = df['grupo'].unique()

    for grupo in grupos:
        info[grupo] = {}
        df_grupo = df[df['grupo'] == grupo]

        metricas = df_grupo['nombre_metric'].unique()

        for metrica in metricas:
            df_met = df_grupo[df_grupo['nombre_metric'] == metrica]
            tipo = df_met['tipo'].iloc[0]

            if tipo == 'numérica':
                valores = df_met['valor_num'].dropna()
                if len(valores) > 0:
                    minimo = valores.min()
                    maximo = valores.max()
                    promedio = valores.mean()
                    info[grupo][metrica] = {
                        'tipo': 'numérica',
                        'min': minimo,
                        'max': maximo,
                        'promedio': promedio
                    }
                else:
                    info[grupo][metrica] = {
                        'tipo': 'numérica',
                        'min': None,
                        'max': None,
                        'promedio': None
                    }
            else:  # categórica
                valores_unicos = df_met['nombre_metric'].unique().tolist()
                info[grupo][metrica] = {
                    'tipo': 'categórica',
                    'valores_unicos': valores_unicos
                }

    return info

# Usalo así:
info_estadisticas = extraer_info_importante(df_estadisticas_procesado)
print(info_estadisticas)


{'GAME STATS': {'Avg. Rating': {'tipo': 'numérica', 'min': np.float64(0.0), 'max': np.float64(8.33), 'promedio': np.float64(5.267333333333333)}, 'No. of Ratings': {'tipo': 'numérica', 'min': np.float64(0.0), 'max': np.float64(44408.0), 'promedio': np.float64(14818.333333333334)}, 'Std. Deviation': {'tipo': 'numérica', 'min': np.float64(0.0), 'max': np.float64(1.65), 'promedio': np.float64(0.9366666666666665)}, 'Weight': {'tipo': 'numérica', 'min': np.float64(1.92), 'max': np.float64(1.92), 'promedio': np.float64(1.92)}, 'Comments': {'tipo': 'numérica', 'min': np.float64(0.0), 'max': np.float64(6141.0), 'promedio': np.float64(2053.3333333333335)}, 'Fans': {'tipo': 'numérica', 'min': np.float64(1.0), 'max': np.float64(2234.0), 'promedio': np.float64(750.3333333333334)}, 'Page Views': {'tipo': 'numérica', 'min': np.float64(3905.0), 'max': np.float64(1852791.0), 'promedio': np.float64(620827.0)}, 'Weight N/A': {'tipo': 'categórica', 'valores_unicos': ['Weight N/A']}}, 'GAME RANKS': {'Overa

In [26]:
from jinja2 import Template
import requests
from decouple import config
from google.colab import userdata

def obtener_filtro_llm(consulta_usuario, df, api_url, headers):

    # Prompt de sistema
    prompt_sistema = """
Eres un asistente experto en manipulación de datos y Pandas.

Tienes la siguiente información sobre métricas agrupadas en categorías:

GAME STATS:
- Avg. Rating: numérica, valores entre 0.0 y 8.33, promedio 5.27
- No. of Ratings: numérica, valores entre 0.0 y 44408.0, promedio 14818.33
- Std. Deviation: numérica, valores entre 0.0 y 1.65, promedio 0.94
- Weight: numérica, valor fijo 1.92
- Comments: numérica, valores entre 0.0 y 6141.0, promedio 2053.33
- Fans: numérica, valores entre 1.0 y 2234.0, promedio 750.33
- Page Views: numérica, valores entre 3905.0 y 1852791.0, promedio 620827.0
- Weight N/A: categórica, valores únicos: ['Weight N/A']

GAME RANKS:
- Overall Rank: numérica, valor fijo 213.0
- Abstract Rank: numérica, valor fijo 10.0
- Family Rank: numérica, valor fijo 45.0
- Overall Rank ‐‐: categórica, valores únicos: ['Overall Rank ‐‐']

PLAY STATS:
- All Time Plays: numérica, valores entre 0.0 y 287803.0, promedio 95935.67
- This Month: numérica, valores entre 0.0 y 512.0, promedio 170.67

COLLECTION STATS:
- Own: numérica, valores entre 7.0 y 73849.0, promedio 24770.0
- Prev. Owned: numérica, valores entre 0.0 y 4914.0, promedio 1651.0
- For Trade: numérica, valores entre 1.0 y 606.0, promedio 204.33
- Want In Trade: numérica, valores entre 2.0 y 832.0, promedio 279.33
- Wishlist: numérica, valores entre 8.0 y 9322.0, promedio 3114.0

PARTS EXCHANGE:
- Has Parts: numérica, valores entre 0.0 y 19.0, promedio 6.33
- Want Parts: numérica, valores entre 0.0 y 19.0, promedio 6.33

La consulta del usuario es: "{consulta_usuario}"

Genera un filtro en Python usando Pandas para aplicar sobre un DataFrame que contiene estas métricas, con el objetivo de obtener los registros que cumplen la consulta.

Responde solo con el código Python del filtro (ejemplo: df[(df['Avg. Rating'] > 7) & (df['Fans'] > 1000)]), sin explicaciones adicionales.
""".replace("{consulta_usuario}", consulta_usuario)

    chat_prompt = [
        {"role": "system", "content": prompt_sistema},
        {"role": "user", "content": f"La consulta del usuario es: '{consulta_usuario}'"}
    ]

    # Jinja template del modelo Zephyr
    template_str = (
        "{% for message in messages %}"
        "{% if message['role'] == 'user' %}<|user|>{{ message['content'] }}</s>\n"
        "{% elif message['role'] == 'assistant' %}<|assistant|>{{ message['content'] }}</s>\n"
        "{% elif message['role'] == 'system' %}<|system|>{{ message['content'] }}</s>\n"
        "{% endif %}{% endfor %}<|assistant|>\n"
    )
    template = Template(template_str)
    prompt_final = template.render(messages=chat_prompt)

    # Armar payload
    data = {
        "inputs": prompt_final,
        "parameters": {
            "max_new_tokens": 128,
            "temperature": 0.5
        }
    }

    # Llamar a la API
    response = requests.post(api_url, headers=headers, json=data)
    generated = response.json()[0]["generated_text"]

    # Extraer filtro
    filtro = generated.split("<|assistant|>")[-1].strip()

    # Aplicar filtro con seguridad usando df.query si es posible
    try:
        df_filtrado = eval(filtro, {"df": df})
        return df_filtrado
    except Exception as e:
        print("Error al aplicar el filtro:", e)
        print("Filtro generado:", filtro)
        return None

# API de HuggingFace con modelo Zephyr
api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"

# Token de Hugging Face desde archivo .env o desde el entorno de Google Colab
api_key = config("HF_TOKEN", default=userdata.get("HF_TOKEN"))

headers = {
    "Authorization": f"Bearer {api_key}"
}

ImportError: cannot import name 'config' from 'decouple' (d:\Documentos del Administrador\Desktop\Facultad\Cuartas Materias\NLP\TP3\tp3_entorno_nlp\lib\site-packages\decouple\__init__.py)

In [ ]:
consulta = "Quiero juegos con más de 2000 fans y un rating mayor a 7"

df_filtrado = obtener_filtro_llm(consulta_usuario=consulta, df=df, api_url=api_url, headers=headers)

print(df_filtrado)


NameError: name 'headers' is not defined